In [15]:
import os
import pandas as pd
import pickle
import json
from pymatgen.io.cif import CifParser
import fnmatch

import warnings
warnings.simplefilter("ignore")

In [16]:
rename = json.load(open("data/rename_map.json", "r"))
mofid_data = pd.read_csv("data/mofid_data.csv", index_col=0)
zeo_data = pd.read_csv("data/zeo_data.csv", index_col=0)

In [81]:
names = list(filter(lambda name: name.endswith(".cif"), os.listdir("cifs")))

In [83]:
len(names)

128

In [18]:
mofid_data.head(3)

,node,linker,cat,mofkey
ed977801beee40131ed6c96cf7ee0193,['[Zn]'],"['N1=CN(CC1)CN1[CH][N]C=C1', '[CH]1[N][CH]N([C...",1.0,Zn.CNLDCJYNBWCRFO.FEEIZMXTBNIYFN.LYBWGVWLQBVNJ...
2a8f3de34fa7366ca48379f94dac4386,['[Zn][O]([Zn][O]([Zn])([Zn])[Zn])([Zn])[Zn]'],"['[O-]C(=O)c1ccc(c(c1[CH])[CH])C(=O)[O-]', '[O...",0.0,Zn.ABMFBCRYHDZLRD.LPFAHSLZEGHMAA.MOFkey-v1
6c0bb3be98debfbcc4849941157d0ef7,['[Cd]'],['N1=C[N]N([CH]1)c1ccc(c2c1nsn2)N1[N]C=N[CH]1'...,1.0,Cd.BMECAHBIWZEJHK.KKEYFWRCBNTPAC.MOFkey-v1


In [19]:
zeo_data.head(3)

,pld,lcd
d990e286a9b8f723557ec5b14c315512,1.89983,0.88674
f09f1b56a897de2c2b0270d53f786bbc,4.44554,1.50468
c946032037f945548b80f2842d5b1678,1.74202,0.85165


In [29]:
len(set(mofid_data.index).intersection(zeo_data.index))

123

In [31]:
bad_parse  = []
bad_get_sg = []

folder = "cifs"
cif_data = pd.DataFrame(columns=["a", "b", "c", "alpha", "beta", "gamma", "volume", "sg_number"])

for file in os.listdir(folder):
    if fnmatch.fnmatch(file, '*.cif'):
        try:
            # print(os.path.join(file))
            struct = CifParser(os.path.join(folder, file)).get_structures()[0]
            try:
                d_lattice = struct.as_dict()['lattice']
                property = [d_lattice["a"], d_lattice["b"], d_lattice["b"],
                            d_lattice["alpha"], d_lattice["beta"], d_lattice["gamma"], d_lattice["volume"],
                            struct.get_space_group_info()[1]]
                cif_data.loc[rename.get(file)[:-4]] = property
            except ValueError:
                print(f"Error with get sg: {os.path.join(file)}")
                bad_get_sg.append(os.path.join(file))
        except ValueError:
            print(f"Error with prase: no structure in {file}")
            bad_parse.append(file)
        except KeyError:
            print(f"Error with prase: no parameters in {file}")
            bad_parse.append(file)

Error with prase: no structure in Er-926.cif
Error with prase: no structure in Er-995.cif
Error with prase: no structure in pvl-457.cif
Error with prase: no structure in pvl-497.cif


In [32]:
cif_data

,a,b,c,alpha,beta,gamma,volume,sg_number
36d2b4157bc92deadc4cb93b125ca7e6,11.1979,14.513000,14.513000,86.68500,70.855000,76.510000,2488.658526,2.0
821b7054efa92e698912933e1fc67f94,11.2003,14.477100,14.477100,86.66700,71.055000,76.476000,2482.517872,2.0
aaf092a5ee04ae8f8d5f19327cf5f925,9.6166,11.589400,11.589400,78.97800,84.243000,84.516000,1490.074819,2.0
c946032037f945548b80f2842d5b1678,6.8540,17.519000,17.519000,96.39000,90.000000,90.000000,3028.098838,14.0
90d2a7cc2d672ea2a68edfdab7f09df0,11.1177,15.073200,15.073200,84.60500,72.423000,76.859000,2578.030965,2.0
...,...,...,...,...,...,...,...,...
a838474e764453afec222b0567fa9679,7.1712,10.281715,10.281715,63.63922,71.242468,71.242468,628.744861,15.0
7f2f52d5940567d37ee4515d60dc5923,6.0826,9.840900,9.840900,79.61900,86.210000,78.351000,889.508696,2.0
0de7676cf802764a602a2785a2e08331,4.2550,14.065400,14.065400,96.46000,90.000000,90.000000,1088.626315,14.0
0ca9173e5ebc8bcea117cdc259d89e2e,12.2206,15.141900,15.141900,90.00000,90.000000,90.000000,3783.002696,61.0


In [34]:
from mordred import Calculator, descriptors
from rdkit import Chem

In [35]:
linkers = [smi.strip('"')[1:-1].replace("'", "").split(", ") for smi in mofid_data['linker'].values]
mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
calc = Calculator(descriptors, ignore_3D=False)
def f(mof):
    try: return calc.pandas(mof)
    except TypeError:
        return None
    
dfs = [f(mof) for mof in mols]

[16:32:45] Can't kekulize mol.  Unkekulized atoms: 19 20 21
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 8 11 12
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 8 11 12
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 8 11 12
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekulize mol.  Unkekulized atoms: 3 4 7
[16:32:45] Can't kekul

C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


100%|██████████| 1/1 [00:02<00:00,  2.43s/it]


C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


...

C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Autocorrelation.py:97: RuntimeWarning: Mean of empty slice.
  return avec - avec.mean()
c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\droid\AppData\Roaming\Python\Python39\site-packages\mordred\Constitutional.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return S / self.mol.GetNumAtoms()


  0%|          | 0/2 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'columns'

In [37]:
data_mordred = pd.DataFrame(columns=dfs[1].columns)

for i, filename in enumerate(mofid_data.index):
    try:
        if linkers[i] != [""]:
            data_mordred.loc[filename] = dfs[i].mean()
    except AttributeError:
        print(f"{filename:_^20}")

ed977801beee40131ed6c96cf7ee0193
b920448ad95f18b8d7a28d89d84fd778
19e12ded00bf6ca6e8bcc777bdb61bc7
386beb547983d357c30ff5ca37a38791
e80fb54ad5770678e0094be5a6c3abd5
59b7436270ce0376b03bec84f6e72a30
821b7054efa92e698912933e1fc67f94
7b50eafdea931f46619e132f85a75578
f21f0642cdb91df0ced81b1113089363
5c770bf0c8835aecaa850ee087b9f62d
7dce4a8dd643623a20ee6311ab2e10f0
3fc1abafbefb4d3b742b9ca94c2c454d
6f822b2cb88a105dfa63d23049732366
4f89e6e2fe55f0e62e38cde8caa7e7c3
5a395f959b3368f251d340afba9372f5
a8f8d0b1c82305f8bbd15e56c8867058
cb20baeed7374110bf9f7d6cdc21757f
608e3ace0a2024c86db7eaa18e60951f
15792c6a1fdf2d04222c02040f7b14a4
f2c8a32861ccb21be9ab6040ea6da391
14b77d18d443b9ee50203dcd37766fbd
36ce01542dca6e2eae577159769efecd
36d2b4157bc92deadc4cb93b125ca7e6
21059adb16277beeeb8e4d937a91cb59
c39f439454e0141cb84760cb3e232010
8e71296ff7bc11fbb260562cabeb33f7
3b9fa4f893c96ca3a8fa674843db95fc
d3aa23631a83f90cd04d59213fbf81da
c946032037f945548b80f2842d5b1678
90d2a7cc2d672ea2a68edfdab7f09df0
e74ac67312

In [54]:
data_mordred.head(3)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
2a8f3de34fa7366ca48379f94dac4386,11.304904,10.377471,2.0,0.0,18.241258,2.418533,4.837066,18.241258,1.213234,3.615821,...,9.643828,47.160462,201.019881,10.086823,344.5,24.0,75.0,88.0,6.666667,3.347222
6c0bb3be98debfbcc4849941157d0ef7,12.098054,10.862691,1.0,2.0,20.158786,2.385982,4.728227,20.158786,1.281786,3.648682,...,9.512946,55.786041,217.027834,10.526249,434.0,21.0,81.0,95.5,4.680556,3.430556
636ad7bfd6a7feec37e3aaf0149fdde7,10.451608,9.626486,0.5,0.0,17.948395,2.206076,4.328873,17.948395,1.255222,3.528870,...,8.776409,52.040879,194.040659,9.351399,499.0,13.0,63.0,66.5,5.152778,3.472222


In [55]:
len(set(mofid_data.index).intersection(zeo_data.index).intersection(data_mordred.index))

75

In [56]:
import numpy as np

def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [60]:
elemental_descriptors = pd.read_csv("../preprocessing/qmof/data/elemental_descriptors.csv")

In [61]:
elemental_descriptors

,Name,Symbol,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
0,Hydrogen,H,1,1.007940,0.53,7.18,0.700,73.0
1,Helium,He,2,4.002602,0.31,0.00,0.198,-50.0
2,Lithium,Li,3,6.941000,1.67,3.01,24.300,60.0
3,Beryllium,Be,4,9.012180,1.12,4.90,5.600,-50.0
4,Boron,B,5,10.811000,0.87,4.29,3.000,27.0
...,...,...,...,...,...,...,...,...
80,Thallium,Tl,81,204.383300,1.56,3.20,7.600,36.0
81,Lead,Pb,82,207.200000,1.54,3.90,6.800,34.0
82,Bismuth,Bi,83,208.980370,1.43,4.69,7.400,91.0
83,Thorium,Th,90,232.038100,1.80,3.20,32.100,113.0


In [62]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid_data.index, mofid_data["node"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with ['*']
Error with ['*']
Error with ['*']
Error with []
Error with ['*']
Error with []
Error with ['*']
Error with []
Error with []
Error with []
Error with []
Error with []


In [65]:
len(set(mofid_data.index).intersection(zeo_data.index).intersection(data_mordred.index).intersection(node_descriptors.index))

75

In [66]:
index = list({*list(zeo_data.index)} & {*list(cif_data.index)} & {*list(data_mordred.index)} & {*list(node_descriptors.index)})

In [69]:
len(index)

74

In [73]:
dataset = pd.concat([zeo_data.loc[index], cif_data.loc[index], data_mordred.loc[index], node_descriptors.loc[index]], axis=1)

In [74]:
dataset

,pld,lcd,a,b,c,alpha,beta,gamma,volume,sg_number,...,mZagreb1,mZagreb2,n_metals,n_types_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
2a8f3de34fa7366ca48379f94dac4386,2.10166,1.18375,15.3786,17.369900,17.369900,90.000000,97.253000,90.000000,5041.515931,14.0,...,6.666667,3.347222,7.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
11ad0e105123bf33c306fc78623eb8be,5.29443,2.37409,15.3990,26.957000,26.957000,108.700000,100.799000,104.920000,10032.378327,2.0,...,5.237847,4.388889,4.0,1.0,48.0,112.41100,1.61,4.33,7.2,-70.0
edf7caa7d500bfc80874598a41c188ba,2.38837,1.19137,12.4021,13.049600,13.049600,83.536000,68.574000,86.544000,1971.676912,2.0,...,6.166667,3.500000,2.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
b9af5ef36ea1a5c04c70b1769492ed84,2.02451,1.11252,7.5543,13.094400,13.094400,90.000000,90.000000,90.000000,1829.804141,29.0,...,4.680556,3.416667,1.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
506bf74aa9636adeaf4c3884557038cd,6.37344,4.27262,9.8840,18.476000,18.476000,113.229000,90.418000,94.136000,3266.368104,2.0,...,4.680556,3.430556,1.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
da1de09efdc4c5c96ae5d881d317a4bf,4.88969,1.76846,12.0467,12.775100,12.775100,106.888000,100.083000,95.076000,2617.777267,2.0,...,4.993056,3.694444,1.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
21508b2b527d68a1b8744b89618a0c1f,2.28758,1.01140,6.7358,14.998100,14.998100,90.000000,94.947000,90.000000,1817.912002,14.0,...,4.680556,3.416667,1.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
6ab6867ac8187471fece36b615703015,2.40730,1.32925,12.4650,13.092000,13.092000,82.815000,67.235000,86.540000,1963.541726,2.0,...,6.166667,3.500000,2.0,1.0,30.0,65.39000,1.42,4.45,6.4,-60.0
03d8c825f033f0be34156906b2a2a191,7.63287,4.40104,14.0590,14.059000,14.059000,69.288468,69.288468,60.000000,3105.679055,161.0,...,4.194444,2.888889,2.0,1.0,25.0,54.93805,1.61,3.72,9.4,-50.0


In [72]:
import joblib
from preproc_model import PreprocessingModel
preproc = joblib.load("../preprocessing/preproc_m.pkl")

In [75]:
x = preproc.transform(dataset.rename({ "sg_number": "spacegroupNumber"}, axis=1))

In [78]:
x.to_csv("x.csv")

TODO
План работы:  
- Сделать предобработку cif-to-vec  
- Сохранить векторизованную базу в csv 
- Через обученные модели (knn, rf, catboost, adaboost) разметить данные  
- Составить таблицу  
Cif name | knn pred | rf pred | catboost pred | adaboost pred | comments  
ВАЖНО: 
- в cif name записать все цифы ,даже те, которые не посчитались 
- Часть структур не получится 100% (не получится mofid, нет металла и пр.)  
- для тех цифов, которые не посчитались так и пиши в таблицу в графу comments - "не посчиталось mofid" ,"нет металла - это COF" и пр. ошибки 

In [119]:
results = pd.DataFrame(columns=["Cif name", "kNN", "RF", "CatBoost", "AdaBoost", "Comments"])
results["Cif name"] = names
results["Hash"] = list(map(lambda n: rename.get(n)[:-4], results["Cif name"]))
results.set_index("Hash", inplace=True)

In [120]:
def set_comments(i: str, comment: str):
    if results.loc[i]["Comments"] is not np.nan:
        results.loc[i]["Comments"] += "/" + comment
    else:
        results.loc[i]["Comments"] = comment


for bad_id in results.index.difference(mofid_data.index):
    set_comments(bad_id, "bad_mofid")
for bad_id in results.index.difference(zeo_data.index):
    set_comments(bad_id, "bad_zeo")
for bad_id in results.index.difference(cif_data.index):
    set_comments(bad_id, "bad_cif")     
for bad_id in results.index.difference(data_mordred.index):
    set_comments(bad_id, "bad_mordred")   
for bad_id in results.index.difference(node_descriptors.index):
    set_comments(bad_id, "bad_node")   

In [121]:
results

,Cif name,kNN,RF,CatBoost,AdaBoost,Comments
Hash,,,,,,
36d2b4157bc92deadc4cb93b125ca7e6,Er-1000.cif,NaN,NaN,NaN,NaN,bad_mordred
821b7054efa92e698912933e1fc67f94,Er-1027.cif,NaN,NaN,NaN,NaN,bad_mordred
aaf092a5ee04ae8f8d5f19327cf5f925,Er-1029.cif,NaN,NaN,NaN,NaN,NaN
c946032037f945548b80f2842d5b1678,er-1065.cif,NaN,NaN,NaN,NaN,bad_mordred
90d2a7cc2d672ea2a68edfdab7f09df0,Er-1085.cif,NaN,NaN,NaN,NaN,bad_mordred
...,...,...,...,...,...,...
a838474e764453afec222b0567fa9679,vvm-129.cif,NaN,NaN,NaN,NaN,bad_mordred/bad_node
7f2f52d5940567d37ee4515d60dc5923,vvm-15.cif,NaN,NaN,NaN,NaN,NaN
0de7676cf802764a602a2785a2e08331,vvm-156.cif,NaN,NaN,NaN,NaN,bad_mordred/bad_node


In [123]:
results.to_csv("results_preproc.csv")